# Brief Métier : Exploitation des Données Scrappées avec Pandas

## Contexte Professionnel

Après la phase de **scraping** réalisée sur des sites concurrents de Castorama, vous avez obtenu deux fichiers CSV :

- categories.csv : contenant les informations relatives aux catégories et sous-catégories.
- products.csv : contenant les informations relatives aux produits (nom, prix, disponibilité, promotions, etc.).

En tant que **Data Analyst / Data Engineer**, votre rôle est désormais de **nettoyer**, **préparer** et **analyser** ces données afin d’en extraire des **informations pertinentes**. Ces informations permettront à Castorama de mieux comprendre l’état du marché, de mettre en place une stratégie tarifaire compétitive et d'anticiper les tendances.

### Installations

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install ipykernel

In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install seaborn

## Importations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Chargement des Données

In [2]:
# Load category data

category_data = pd.read_csv("castorama_categories.csv")

# Load products data

product_data = pd.read_csv("castorama_products.csv")

In [ ]:
# Confirm category data loaded correctly

category_data

In [ ]:
# Confirm product data loaded correctly

product_data

## Exploration et Premières Manipulations

### Aperçu des données

In [ ]:
# Get information about category_data

category_data.info()

In [ ]:
# Get information about product_data

product_data.info()

In [ ]:
# View category_data summary statistics 

category_data.describe()

In [ ]:
# View product_data summary statistics 

product_data.describe()

In [ ]:
# View first 5 data in category_data

category_data.head()

In [ ]:
# View first 5 data in product_data

product_data.head()

## Nettoyage et Préparation des Données

### Category_data.csv

In [3]:
# Duplicate raw data

cdf = category_data.copy()

In [ ]:
# Check for missing data

cdf.isna().sum()

In [6]:
# View duplicated categories 

pd.set_option('display.max_rows', None)

duplicates = cdf[cdf["category"].duplicated(keep=False)]

In [ ]:
# Sort duplicate data for further understanding

pd.set_option('display.max_colwidth', None)

duplicates.sort_values(by='url')

In [4]:
# Drop duplicates (keep only first occurrence)

cdf.drop_duplicates(subset=["category"], inplace=True)

# View data
cdf.describe()

,category,is_page_list,url
count,1871,1871,1871
unique,1871,2,1384
top,arrosage-enterre,False,https://www.castorama.fr/peinture/peinture-int...
freq,1,1371,3


In [ ]:
# View duplicated urls

duplicated_urls = cdf[cdf["url"].duplicated(keep=False)]

# Sort by url
duplicates_sorted = duplicated_urls.sort_values(by="url")

# View data
duplicates_sorted

In [5]:
# Drop duplicate urls (Observation: Double is_page_list created for SEO and Diacritics)

cdf.drop_duplicates(subset=["url"], inplace=True)

# Summarize data
cdf.describe()

,category,is_page_list,url
count,1384,1384,1384
unique,1384,2,1384
top,Serre de jardin,False,https://www.castorama.fr/jardin-et-terrasse/se...
freq,1,1369,1


- Manipulation de chaînes :

In [6]:
# Remove trailing spaces and characters in category name

cdf["category"] = cdf["category"].str.strip()

In [7]:
# Convert category names to lowercase

cdf['category'] = cdf['category'].str.lower()

In [8]:
# Handling encoding issues (trailing underscores, Prefix 0s, multiple underscores) in specified columns
# Replace spaces, commas, apostrophes with underscore

import re

def clean_text(input_str):
    if pd.isnull(input_str):  # Handle NaN values
        return input_str
    input_str = str(input_str)  # Ensure the input is a string
    input_str = re.sub(r'[\u002D\u2010\u2011\u2012\u2013\u2014\u2212]', '_', input_str) # Replace all hyphen types
    input_str = re.sub(r'\s+', '_', input_str.strip())  # Replace all whitespace with underscores
    input_str = input_str.replace(',', '_')  # Replace commas with underscores
    input_str = input_str.replace("'", '_')  # Replace apostrophes with underscores
    input_str = input_str.replace('\xa0', '_')  # Replace non-breaking spaces
    input_str = re.sub(r'_+', '_', input_str)  # Remove multiple underscores
    input_str = re.sub(r'^_|_$', '', input_str)  # Remove leading or trailing underscores
    input_str = re.sub(r'^0+', '', input_str)  # Remove leading zeros
    return input_str

cdf['category'] = cdf['category'].map(clean_text)

In [9]:
# Remove accents

replacements = {"à": "a", "á": "a", "â": "a", "ä": "a",
                "é": "e", "è": "e", "ê": "e", "ë": "e", "É":"E", "È":"E",
                "î": "i", "ï":"i", "ì": "i", "í": "i",
                "ö": "o", "ô": "o", "ò": "o", "ó": "o",
                "ü": "u", "û": "u", "ù": "u", "ú": "u"}

def replace_accents(input_str, replacement):
    for old, new in replacement.items():
        input_str = input_str.replace(old, new)
    return input_str

cdf["category"] = cdf["category"].apply(lambda x: replace_accents(str(x), replacements))

In [ ]:
# Review for errors or duplicates

cdf.sort_values(by='category')

In [11]:
# Verify if duplicates persists

c_duplicates = cdf[cdf['category'].duplicated(keep=False)]

c_duplicates.sort_values(by='category')

,category,is_page_list,url
1483,accessoire_luminaire,False,https://www.castorama.frhttps://www.castorama....
1872,accessoire_luminaire,True,https://www.castorama.fr/luminaire-et-eclairag...
882,brosse_wc,True,https://www.castorama.fr/salle-de-bains-et-wc/...
541,brosse_wc,False,https://www.castorama.frhttps://www.castorama....
609,colonne_de_douche_hydromassante,False,https://www.castorama.frhttps://www.castorama....
849,colonne_de_douche_hydromassante,True,https://www.castorama.fr/salle-de-bains-et-wc/...
891,colonne_de_douche_pour_baignoire,True,https://www.castorama.fr/salle-de-bains-et-wc/...
614,colonne_de_douche_pour_baignoire,False,https://www.castorama.frhttps://www.castorama....
879,douchette_wc,True,https://www.castorama.fr/salle-de-bains-et-wc/...
542,douchette_wc,False,https://www.castorama.frhttps://www.castorama....


In [24]:
# Sort by is_page_list

cdf_sorted = cdf.sort_values(by="is_page_list", ascending=False)

In [25]:
# Drop newly found duplicates (SEO / Diacritics related, keep only "True" is_page_lists)

cdf_no_duplicates = cdf_sorted.drop_duplicates(subset=['category'], keep='first')

In [ ]:
pd.set_option('display.max_rows', None)

cdf_no_duplicates.sort_values(by='category')

In [ ]:
cdf_no_duplicates.reset_index(drop=True)

### Product_data.csv

In [ ]:
# Duplicate raw product data

pdf = product_data.copy()

In [ ]:
# Get info about products data

pdf.info()

In [ ]:
# Get summary statistics/info

pdf.describe()

In [ ]:
# Check for missing values (general)

pdf.isna()

In [ ]:
# Check for missing values in unique_id column

pdf["unique_id"].isna().value_counts()

In [ ]:
# Check for missing values (category column)

pdf["category"].isna().value_counts()

In [ ]:
# Check for missing values (subcategory column)

pdf["subcategory"].isna().value_counts()

In [ ]:
# Check for missing values (subsubcategory column)

pdf["subsubcategory"].isna().value_counts()

In [ ]:
# Check for missing values (subsubsubcategory column)

pdf["subsubsubcategory"].isna().value_counts()

In [ ]:
# Check for missing values (price column)

pdf["price"].isna().value_counts()

In [ ]:
# Check for missing values (title column) 

pdf["title"].isna().value_counts()

In [ ]:
# Check for missing values (url column)

pdf["url"].isna().value_counts()

- Manipulation de chaînes :

In [ ]:
# Replace NaNs

pdf['subsubsubcategory'] = pdf['subsubsubcategory'].fillna("Not_available")
pdf

In [ ]:
# Convert specified columns to lowercase 

columns_to_lowercase = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

pdf[columns_to_lowercase] = pdf[columns_to_lowercase].apply(lambda x: x.str.lower())

pdf

In [ ]:
# Strip values in the specified columns 

columns_to_strip = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

pdf[columns_to_strip] = pdf[columns_to_strip].apply(lambda x: x.str.strip())

pdf

In [ ]:
# Handling encoding issues (trailing underscores, Prefix 0s, multiple underscores) in specified columns
# Replace spaces, commas, apostrophes with underscore

import re

columns_to_replace = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

def clean_text(input_str):
    if pd.isnull(input_str):  # Handle NaN values
        return input_str
    input_str = str(input_str)  # Ensure the input is a string
    input_str = re.sub(r'[\u002D\u2010\u2011\u2012\u2013\u2014\u2212]', '_', input_str) # Replace all hyphen types
    input_str = re.sub(r'\s+', '_', input_str.strip())  # Replace all whitespace with underscores
    input_str = input_str.replace(',', '_')  # Replace commas with underscores
    input_str = input_str.replace("'", '_')  # Replace apostrophes with underscores
    input_str = re.sub(r'_+', '_', input_str)  # Remove multiple underscores
    input_str = re.sub(r'^_|_$', '', input_str)  # Remove leading or trailing underscores
    input_str = re.sub(r'^0+', '', input_str)  # Remove leading zeros
    return input_str

pdf[columns_to_replace] = pdf[columns_to_replace].map(clean_text)
pdf

In [ ]:
# Replace accented letters in the specified columns' texts

import unicodedata

columns_to_replace = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

def robust_remove_accents(input_str):
    # Normalize to decomposed form
    normalized = unicodedata.normalize('NFD', input_str)
    # Remove combining characters (accents)
    without_accents = ''.join(c for c in normalized if unicodedata.category(c) != 'Mn')

    # Explicitly replace problematic characters (if any remains)
    replacements = {"à": "a", "á": "a", "â": "a", "ä": "a",
                "é": "e", "è": "e", "ê": "e", "ë": "e", "É":"E", "È":"E",
                "î": "i", "ï":"i", "ì": "i", "í": "i",
                "ö": "o", "ô": "o", "ò": "o", "ó": "o",
                "ü": "u", "û": "u", "ù": "u", "ú": "u"}

    for accented_char, replacement in replacements.items():
        without_accents = without_accents.replace(accented_char, replacement)
    
    # Handle lingering issues and strip
    return without_accents.replace('\xa0', ' ').strip()

pdf[columns_to_replace] = pdf[columns_to_replace].map(
    lambda x: robust_remove_accents(str(x)) if isinstance(x, str) else x)

pdf

In [ ]:
# Convert price column to float type

pdf['price'] = pdf['price'].apply(lambda x: x.replace(",", "."))
pdf['price'] = pdf['price'].apply(lambda x: x.replace(" ", ""))

pdf['price'] = pd.to_numeric(pdf['price'], errors='coerce')

In [ ]:
# Display count of prices with NaN

pdf['price'].isna().value_counts()

In [ ]:
# Show rows with NaNs to understand the problem

pdf_nas = pdf[pdf.isna().any(axis=1)]

pdf_nas

In [41]:
# Replace NaNs with None

pdf['price'] = pdf['price'].replace({pd.NA: None, np.nan: None})

# Drop rows with NaN
pdf = pdf.dropna(subset=['price'])

In [ ]:
# Reset index

pdf.reset_index(drop=True)